In [44]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [45]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [151]:
# skip if result.txt exists
y16 = glob.glob('data/2016'+'/*.txt')
y15 = glob.glob('data/2015'+'/*.txt')
y14 = glob.glob('data/2014'+'/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==9:
                    outfile.write(line+'\n')

In [152]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author']
df.shape

(9043461, 9)

In [153]:
#df=df[df['author']==1]
#df.shape

In [154]:
df['year']=df['year'].astype(int)

In [155]:
df=df[df['year']<=2017][df['year']>=1950]
df.shape

(9043461, 9)

In [156]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [157]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(9038505, 12)

In [158]:
df=df[['count','cited','citation','issnp','issne','journal']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [159]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [160]:
s=df.issn.str.split(',',expand=True).stack()

In [161]:
i = s.index.get_level_values(0)

In [162]:
df2 = df.loc[i].copy()

In [176]:
df2["@id"] = s.values

In [177]:
dg=df2.groupby(['@id']).agg({'count':sum,'cited':sum,'citation':sum,'journal':max}).reset_index()
dg.shape

(58863, 5)

In [178]:
dg2=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()

In [179]:
dg['h']=dg2.citation.apply(lambda x: h_(x))

In [180]:
dg.sort_values('h',ascending=False).head()

,@id,count,cited,citation,journal,h
19811,1533-4406,3353,2673,128792,New England Journal of Medicine,178
1782,0028-4793,3353,2673,128792,New England Journal of Medicine,178
18344,1476-4687,12087,5719,186122,NatureJobs,173
1760,0028-0836,10125,5509,185754,Nature,173
2266,0036-8075,13689,4788,168177,Science,164


In [181]:
dg['impact']=dg['citation']/dg['count']
dg.sort_values('impact',ascending=False).head()

,@id,count,cited,citation,journal,h,impact
447,0007-9235,146,96,19417,CA: A Cancer Journal for Clinicians,26,132.993151
19097,1520-6890,958,890,76926,Chemical Reviews,152,80.298539
525,0009-2665,958,890,76926,Chemical Reviews,152,80.298539
16565,1433-8351,13,12,984,Living Reviews in Relativity,7,75.692308
52758,2367-3613,13,12,984,Living Reviews in Relativity,7,75.692308


issnps_all=df.issnp.values
df.loc[(df['issne'].isin(issnps_all)) & (df['issnp']==''),'issnp']=df['issne']
issnps=df[(df.issne!='') & (df.issnp!='')].issnp.values
issnes=df[(df.issne!='') & (df.issnp!='')].issne.values
p={}
i=0
for issnp in issnps:
    p[issnp]=issnes[i]
    i=i+1
e={}
i=0
for issne in issnes:
    e[issne]=issnps[i]
    i=i+1
df['issnp2']=df.apply(lambda x: e[x['issne']] if x['issne'] in e else x['issnp'],axis=1)
df['issne2']=df.apply(lambda x: p[x['issnp']] if x['issnp'] in p else x['issne'],axis=1)


In [772]:
dg=df.groupby(['issnp2','issne2']).agg({'count':sum,'cited':sum,'citation':sum,'journal':max}).reset_index()
dg.shape

(40130, 6)

In [773]:
dg2=df[['issnp2','issne2','citation']].groupby(['issnp2','issne2']).agg(lambda x:tuple(x)).reset_index()
dg['h']=dg2.citation.apply(lambda x: h_(x))

In [778]:
dg.sort_values('h',ascending=False).head()

,issnp2,issne2,count,cited,citation,journal,h,impact,@id
9839,0028-4793,1533-4406,3353,2673,128792,New England Journal of Medicine,178,38.410975,0028-4793
9817,0028-0836,1476-4687,12088,5720,186123,NatureJobs,173,15.397336,0028-0836
10310,0036-8075,1095-9203,14342,4802,168192,Science,164,11.727235,0036-8075
8594,0009-2665,1520-6890,958,890,76926,Chemical Reviews,152,80.298539,0009-2665
13819,0306-0012,1460-4744,1377,1074,82084,Chemical Society Reviews,149,59.610748,0306-0012


In [775]:
dg['impact']=dg['citation']/dg['count']
dg.sort_values('impact',ascending=False).head()

,issnp2,issne2,count,cited,citation,journal,h,impact
8516,0007-9235,,146,96,19417,CA: A Cancer Journal for Clinicians,26,132.993151
8594,0009-2665,1520-6890,958,890,76926,Chemical Reviews,152,80.298539
38328,2367-3613,1433-8351,13,12,984,Living Reviews in Relativity,7,75.692308
13819,0306-0012,1460-4744,1377,1074,82084,Chemical Society Reviews,149,59.610748
10199,0034-6861,1539-0756,124,120,7322,Reviews of Modern Physics,47,59.048387


In [776]:
dg['@id']=dg.apply(lambda x:x['issne2'] if x['issnp2']=='' else x['issnp2'],1)

In [182]:
dic=[]
for index, i in dg.iterrows():
    journal={}
    journal['@id']=i['@id']
    #issn_list=[i['@id']]
    #if (i['issne2']!='') and (i['issne2']!=i['@id']):
        #issn_list.append(i['issne2'])
    #journal['e:issn']=issn_list
    journal['e:count']=i['count']
    journal['e:cited']=i['cited']
    journal['e:citation']=i['citation']
    journal['e:h']=i['h']
    journal['e:impact']=round(i['impact'],2)
    if journal not in dic:
        dic.append(journal)

In [183]:
with open('journals_stats.jsonld', 'w') as outfile:
    json.dump(dic, outfile)

In [184]:
dic[0]

{'@id': '0000-992X',
 'e:citation': 0L,
 'e:cited': 0L,
 'e:count': 4L,
 'e:h': 0L,
 'e:impact': 0.0}